In [10]:
%run TEST/WZX/Test/send_email

StatementMeta(, 382, -1, Finished, Available)

请设置receiver_email、subject、body，以发送邮件


In [4]:
# 导包
from notebookutils import mssparkutils  
import pandas as pd
import os
from datetime import datetime,timedelta    

StatementMeta(aasparkpool001, 882, 5, Finished, Available)

In [5]:
receiver_email = ['external.Zhaoxiang.Wang@cn.bosch.com','zhaoxiang_wang@leansight.cn','fei.zhao2@cn.bosch.com'] # set email 
subject = 'Bosch Job Alarm: Travel Expense'
body = 'Travel Expense 差旅数据未更新, 请及时确认————This is an alarm email from python. Please do not reply'

StatementMeta(aasparkpool001, 882, 6, Finished, Available)

In [6]:
mssparkutils.fs.unmount('/mnt/bosch-dw-integration-layer')

StatementMeta(aasparkpool001, 882, 7, Finished, Available)

True

In [7]:
# 挂载blob
mssparkutils.fs.mount(
    "wasbs://bosch-dw-integration-layer@proddataplatcn3blob01.blob.core.chinacloudapi.cn",
    "/mnt/bosch-dw-integration-layer",
    {"LinkedService" : "AzureBlobStorage_proddataplatcn3blob01"}
)

path = mssparkutils.fs.getMountPath("/mnt/bosch-dw-integration-layer")

StatementMeta(aasparkpool001, 882, 8, Finished, Available)

In [29]:
print('==================travel expense: Travel=====================')
def get_last_month():
    """ 
     * @ message : 获取上月与本月月份名
     * @ return   [list] 月份
    """
    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.month
    previous_month = current_month - 1 if current_month > 1 else 12
    previous_year = current_year if current_month > 1 else current_year -1
    last_month = str(previous_year) + str(previous_month).zfill(2)
    now_month = str(current_year) + str(current_month).zfill(2)
    month_list = [last_month] # remove current month
    return month_list


def data_need(blobs):
    """ 
     * @ message : 获取需要的blob
     * @ param2   [type] blobs: blobs
     * @ return   [type]
    """
    need_blobs = []
    for i in blobs:
        split_blob = i.split('_')
        if len(split_blob) > 2:
            blob_name = i.split('_')[-2][:6]
        else:
            blob_name = i
        if blob_name not in get_last_month():
            continue
        need_blobs.append(i)
        
    return need_blobs

def hand_file():
    path = mssparkutils.fs.getMountPath("/mnt/bosch-dw-integration-layer")
    root = path + '/manual/AA_CTG/Travel Appication'
    files = os.listdir(root)
    files = data_need(files) # enble filter
    try:
        file_path = os.path.join(root,files[0])
        print(file_path)
        return file_path
    except:
        print('=======未识别到上月差旅数据=========')
        send_to_email(receiver_email, subject, body)
        mssparkutils.notebook.exit()
file_path = hand_file()

StatementMeta(aasparkpool001, 882, 30, Finished, Available)

==================travel expense: Travel=====================
/synfs/882/mnt/bosch-dw-integration-layer/manual/AA_CTG/Travel Appication/travel_application_20231201_20231231.xlsx


In [ ]:
# 处理数据
df_pd = pd.read_excel(file_path,dtype=str)
df_pd = df_pd.dropna(axis=1,how="all")  # 去掉整列都是空值的列

df_pd.columns = df_pd.iloc[2]       
df = df_pd.iloc[3:-1,:]

for col in df.columns:  # 转换nan 列名
    if pd.isna(col):
        df = df.rename(columns ={col:'trip_itinerary'})   


ods_path = 'abfss://data-warehouse-ods@dlsaaddpnorth3001.dfs.core.chinacloudapi.cn/ods_mau_travel_app_cn_mf.csv' # path
di_path = 'abfss://data-warehouse-ods@dlsaaddpnorth3001.dfs.core.chinacloudapi.cn/ods_mau_travel_app_cn_mi.csv' # path
formatted_endtime = datetime.now()
formatted_endtime += timedelta(hours=8)
etl_load_time = formatted_endtime.strftime(f"%Y-%m-%d %H:%M:%S")    # etl_load_time

df1 = df
df1['etl_load_time'] = etl_load_time
df1.to_csv(di_path,index=False, header=True)    # 保持数据格式一致性

df1 = pd.read_csv(di_path,dtype=str).drop('etl_load_time',axis=1) 
ods_df = pd.read_csv(ods_path,dtype=str).drop('etl_load_time',axis=1) # read

try:
    result = pd.concat([ods_df, df1]).reset_index(drop=True).drop_duplicates()  # handle
except Exception as e:
    print(f'===========源数据增加列{e}============')
    # send_to_email(receiver_email, subject, body1)
    mssparkutils.notebook.exit()

result['etl_load_time'] = etl_load_time

mssparkutils.fs.rm(di_path)
result.to_csv(ods_path,index=False, header=True) # save
